In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

11501568/11490434 [==============================] - 2s 0us/step


In [3]:
def custom_mean_squared_error(y_true, y_pred):
    """自定义损失函数方式一"""
    return tf.math.reduce_mean(tf.square(y_true - y_pred))


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=5)

Epoch 1/5
782/782 [==============================] - 2s 1ms/step - loss: 0.0160
Epoch 2/5
782/782 [==============================] - 1s 1ms/step - loss: 0.0074
Epoch 3/5
782/782 [==============================] - 1s 1ms/step - loss: 0.0055
Epoch 4/5
782/782 [==============================] - 1s 1ms/step - loss: 0.0045
Epoch 5/5
782/782 [==============================] - 1s 2ms/step - loss: 0.0038


In [4]:
class CustomMSE(keras.losses.Loss):  # 继承自:tf.keras.losses.Loss
    """自定义损失函数方式二"""

    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)  # ★★★★★不能省略

        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=5)

Epoch 1/5
782/782 [==============================] - 2s 1ms/step - loss: 0.0384
Epoch 2/5
782/782 [==============================] - 1s 2ms/step - loss: 0.0313
Epoch 3/5
782/782 [==============================] - 1s 1ms/step - loss: 0.0296
Epoch 4/5
782/782 [==============================] - 1s 1ms/step - loss: 0.0288
Epoch 5/5
782/782 [==============================] - 1s 1ms/step - loss: 0.0282
